In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import os
import librosa
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 
from pathlib import Path
import IPython.display as ipd#ses dosyasını oynatmak için
from sklearn.metrics import accuracy_score


In [ ]:
audio_file_path='deneme.wav';

In [ ]:
#ses dosyasını dijitalleştirme
librosa_audio_data, librosa_sample_rate = librosa.load(audio_file_path)
librosa_audio_data

In [ ]:
#ses dosyasını görselleştirme mono
plt.figure(figsize=(12, 4))
plt.plot(librosa_audio_data)
plt.show()

In [ ]:
#ses scipy ile okursak iki boyutlu okuyabiliriz sterio
from scipy.io import wavfile as wav
wave_sample_rate, wave_audio = wav.read(audio_file_path)

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(wave_audio)
plt.show()

In [ ]:
#sesimizden mfcc özelliklerini çıkardık.Ses dosyasını işlenir hale getirdik
mfccs = librosa.feature.mfcc(y=librosa_audio_data, sr=librosa_sample_rate, n_mfcc=40)  #40 çıkarılan özellik sayısı
mfccs.shape

In [ ]:
mfccs

In [ ]:
metadata=pd.read_csv('output.csv')
#mp3 yazan yerleri waw ile değiştirdik 
metadata["path"] = metadata["path"].apply(lambda x: x.replace("mp3", "wav"))
metadata.head(10)

In [ ]:
#okunan ses dosyasını dijitalleştiren özellik çıkartan ve normalize eden fonksiyon
def features_extractor(filename):
    audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_scaled_features

In [ ]:
#verisetindeki dosya adlarını, yol olarak kullanıp ses dosyalarını okuyup,dijitalleştirip 
#yeni bir df de ses dosyası ve sınıf etikini birleşiriyoruz
audio_dataset_path='WAV'
extracted_features=[]
for index_num,row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path),str(row["path"]))#WAV/common_voice_tr_23660893.wav
    final_class_labels=row["gender"]
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])

In [ ]:
#yeni dfnin columnslarını isimlendiriyoruz
extracted_features_df = pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()

In [ ]:
metadata["gender"].isnull().sum()#nan satırları sayma

In [ ]:
#ohter sınıf etikiti olan verisi nan değere çeviriyoruz
extracted_features_df['class']=extracted_features_df['class'].replace("other",np.NaN)

In [ ]:
#nan değerleri kaldırıyoruz
extracted_features_df.dropna(inplace=True)

In [ ]:
extracted_features_df["class"].value_counts()#sınıf etiketlerinin türleri ve sayısını buluyoruz.

In [ ]:
extracted_features_df.sort_values("class",ignore_index=True,inplace=True)#sınıf etiketlerini azdan çoğa sıralıyoruz

In [ ]:
extracted_features_df

In [ ]:
#erkek ve kadın örnekleri eşit olması için erkek örnekleri dften çıkarıyoruz
for i in range(21964,3623,-1):
    extracted_features_df.drop([i],axis=0,inplace=True)    
    

In [ ]:
extracted_features_df["class"].value_counts()

In [ ]:
extracted_features_df.to_csv('veriseti.csv')

In [ ]:
# özellikler ve sınıf etiketlerini ayırma
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

In [ ]:
y

In [ ]:
#çıktı için one hot encoding
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))


In [ ]:
y

In [ ]:
#train test slit
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
y_test

In [ ]:
num_labels=2

In [ ]:
# CNN model oluşturma

model=Sequential()
# 1. hidden layer
model.add(Dense(125,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
# 2. hidden layer
model.add(Dense(250))
model.add(Activation('relu'))
model.add(Dropout(0.5))
# 3. hidden layer
model.add(Dense(125))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# output layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [ ]:
model.summary() #model özeli

In [ ]:
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')

In [ ]:
# Modeli Eğitme
model.fit(X_train, y_train, batch_size=32, epochs=300, validation_data=(X_test, y_test), verbose=1)

In [ ]:
# başarı oranı
validation_test_set_accuracy = model.evaluate(X_test,y_test,verbose=0)
print(validation_test_set_accuracy[1])

In [ ]:
#Örnekle deneme
#filename="deneme sesleri/ahmet1.wav"
#sound_signal, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
#mfccs_features = librosa.feature.mfcc(y=sound_signal, sr=sample_rate, n_mfcc=40)
#mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

In [ ]:
#verilen ses dosyası
#ipd.Audio(filename)
#ipd.Audio(sound_signal,rate=sample_rate)

In [ ]:
#waw formatında ses kayıt etme
import pyaudio#ses kaydetmek için
import wave

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "deneme sesleri/kaydedilen.wav"
p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* kaydediyor")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("* kayıt tamamlandı")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

In [ ]:
#kaydedin ses dosyası
ipd.Audio(WAVE_OUTPUT_FILENAME)
librosa_audio_data, librosa_sample_rate = librosa.load(WAVE_OUTPUT_FILENAME)
ipd.Audio(librosa_audio_data,rate=librosa_sample_rate)

In [ ]:
#kaydedilen ses dosyasını görselleştirme
plt.figure(figsize=(12, 4))
plt.plot(librosa_audio_data)
plt.show()

In [ ]:
filename=WAVE_OUTPUT_FILENAME
sound_signal, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
mfccs_features = librosa.feature.mfcc(y=sound_signal, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

In [ ]:
mfccs_scaled_features = mfccs_scaled_features.reshape(1,-1)

In [ ]:
result_array = model.predict(mfccs_scaled_features)

In [ ]:
result_array

In [ ]:
result_classes = ["Kadın","Erkek"]

result = np.argmax(result_array[0])
print(result_classes[result]) 